# Original model prediction

In [ ]:
#Run script path
"/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/scripts/Original_prediction.py"

# data

In [7]:
def fix_name(path,save_path):
    data=pd.read_csv(path)
    data=data[['CDR3B','Epitope','Affinity']]
    data.rename(columns={'CDR3B': 'cdr3', 'Epitope': 'epi', 'Affinity': 'y_true'}, inplace=True)
    data.to_csv(save_path) 

In [8]:
import pandas as pd
database=['Dean','TCRdb']
name=['1000','5000','10000','100000','1000000']
for i in name:
    path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/time/data_"+i+".csv"
    save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/time/data_"+i+".csv"
    fix_name(path,save_path)

# Model retraining

In [1]:
import os
GPU_NUMBER = [0]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO"
import sys
sys.path.append('.')
import numpy as np
import pandas as pd
import shutil
import argparse
from tqdm import tqdm
import pytorch_lightning as pl
pl.seed_everything(0)
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from scripts.model_raw import TEIM
from utils.misc import load_config, calc_auc_aupr
from utils.dataset import load_data, SeqLevelDataset
import os


class SeqLevelSystem(pl.LightningModule):
    def __init__(self, config, train_set, val_set):
        super().__init__()
        self.config = config
        self.lr = config.training.lr
        self.teim_seq = TEIM(config.model)
        self.train_set = train_set
        self.val_set = val_set
        
    
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.config.training.batch_size, shuffle=True)
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.config.training.batch_size, shuffle=False)

    def forward(self, x):
        return self.teim_seq(x)['seqlevel_out']
    
    def minimum_step(self, batch, device=None):
        # batch = batch.to(self.device)
        if device is None:
            cdr3, epi, labels = batch['cdr3'], batch['epi'], batch['labels']
        else:
            cdr3, epi, labels = batch['cdr3'].to(device), batch['epi'].to(device), batch['labels'].to(device)
        pred = self([cdr3, epi])
        loss = self.get_loss(pred, labels)
        return loss, labels, pred

    def training_step(self, batch, batch_idx):
        self.train()
        loss, labels, pred = self.minimum_step(batch)
        self.log('train/loss', loss)
        
        return {
            'loss':loss,
            'labels': labels,
            'pred': pred
        }

    def training_epoch_end(self, training_step_outputs):
        
        ## training metric
        loss, auc, aupr, auc_mean, aupr_mean = self.evaluate_model(self.train_dataloader())

        print('Train set: AUC={:.4}, AUPR={:.4}, AUC_AVG={:.4}, AUPR_AVG={:.4}'.format(auc, aupr, auc_mean, aupr_mean))
        self.log('lr', self.optimizers().state_dict()['param_groups'][0]['lr'])
        self.log_dict({
            'train/auc':auc,
            'train/aupr':aupr,
            'train/auc_avg':auc_mean,
            'train/aupr_avg':aupr_mean,
        }, prog_bar=False)

        

        ## validating metric
        loss, auc, aupr, auc_mean, aupr_mean = self.evaluate_model(self.val_dataloader())
        print('Valid', ' set: AUC={:.4}, AUPR={:.4}, AUC_AVG={:.4}, AUPR_AVG={:.4}'.format(auc, aupr, auc_mean, aupr_mean))
        self.log_dict({
            'valid/loss':loss,
            'valid/auc':auc,
            'valid/aupr':aupr,
            'valid/auc_avg':auc_mean,
            'valid/aupr_avg':aupr_mean,
        }, prog_bar=False)


    def evaluate_model(self, data_loader=None, ):
        self.eval()
        loss = 0
        y_true, y_pred = [], []
        epi_ids = []

        for i, batch in enumerate(data_loader):
            loss_this, y, y_hat = self.minimum_step(batch, self.device)
            loss += loss_this.item()
            y_true.extend(y.cpu().numpy().tolist())
            y_pred.extend(y_hat.detach().cpu().numpy().tolist())
            if 'epi_id' in batch:
                epi_ids.extend(batch['epi_id'].cpu().numpy().tolist())
        loss /= (i+1)
        auc, aupr = self.get_scores(y_true, y_pred)
        print(auc)
        print(aupr)
        ## per epi auc
        if len(epi_ids) > 0:
            ids_uni = np.unique(epi_ids, axis=0)
            print(ids_uni.shape)

            auc_sum = 0
            aupr_sum = 0
            cnt = 0
            for i, id_ in enumerate(ids_uni):
                index = np.array(epi_ids == id_)
                y_true_epi = np.array(y_true)[index]
                y_pred_epi = np.array(y_pred)[index]
                auc_epi, aupr_epi = self.get_scores(y_true_epi, y_pred_epi)
                if auc_epi is None:
                    continue
                auc_sum += auc_epi
                aupr_sum += aupr_epi
                cnt += 1
            auc_mean = auc_sum / cnt
            aupr_mean = aupr_sum / cnt
        else:
            auc_mean, aupr_mean = auc, aupr

        return loss, auc, aupr, auc_mean, aupr_mean


    def predict(self, data_loader=None):
        self.eval()
        cdr3_seqs, epi_seqs, y_true, y_pred = [], [], [], []
        epi_ids = []

        for i, batch in tqdm(enumerate(data_loader), desc='Predicting'):
            loss, y, y_hat = self.minimum_step(batch, self.device)
            cdr3_seqs.extend(batch['cdr3_seqs'])
            epi_seqs.extend(batch['epi_seqs'])
            y_true.extend(y.cpu().numpy().tolist())
            y_pred.extend(y_hat.detach().cpu().numpy().tolist())
            if 'epi_id' in batch.keys():
                epi_ids.extend(batch['epi_id'].cpu().numpy().tolist())

        if len(epi_ids) > 0:
            return cdr3_seqs, epi_seqs, y_true, np.reshape(y_pred, -1), epi_ids
        else:
            return cdr3_seqs, epi_seqs, y_true, np.reshape(y_pred, -1)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def get_loss(self, pred, labels):
        loss = F.binary_cross_entropy(pred.view(-1), labels.float(), weight=None, reduction='mean')
        return loss

    def get_scores(self, y_true, y_pred):
        if len(np.unique(y_true)) == 1:
            return None, None
        else:
            return calc_auc_aupr(y_true, y_pred)

Global seed set to 0


In [2]:
config_path = 'configs/seqlevel_all.yml'
config = load_config(config_path)

In [3]:
def train_main(trainfile_path, testfile_path, train_name, test_name, model_name,save_model_path, result_path):
    class Config:
        def __init__(self, path, file_list):
            self.path = path
            self.file_list = file_list

    config_path = 'configs/seqlevel_all.yml'
    config = load_config(config_path)  

    train_set = SeqLevelDataset(Config(path=trainfile_path, file_list=[train_name])) 
    val_set = SeqLevelDataset(Config(path=testfile_path, file_list=[test_name])) 

    model = SeqLevelSystem(config, train_set, val_set)
    checkpoint = ModelCheckpoint(monitor='valid/auc_avg', save_last=True, mode='max', save_top_k=1)
    earlystop = EarlyStopping(monitor='valid/auc_avg', patience=15, mode='max')
    trainer = pl.Trainer(
        max_epochs=config.training.epochs,
        gpus=1,
        callbacks=[checkpoint, earlystop],
        default_root_dir=os.path.join(os.getcwd(), 'logs', config.name)
    )

    trainer.fit(model)
    model_save_path = os.path.join(save_model_path, f'{model_name}_model.pth')
    torch.save(model.state_dict(), model_save_path)
    shutil.copy2(config_path, os.path.join(trainer.log_dir, os.path.basename(config_path)))

    print('Predicting and saving results...')
    results = model.predict(model.val_dataloader())
    columns = ['CDR3B', 'Epitope', 'y_true', 'y_prob']
    if len(results) == 4:
        pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob']).to_csv(result_path+'probability.csv', index=False)
    else:
        aa= pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob', 'epi_id'])
        aa=aa[['CDR3B', 'Epitope', 'y_true', 'y_prob']]
        aa['y_pred'] = aa['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
        aa.to_csv(result_path+'probability.csv', index=False)
    print('Done')


# pair50

In [27]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1test'
            model_name=k+'_'+j
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/seen/test/"+i+"/"+k+'_'+j
            save_model_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"
            train_main(trainfile_path,testfile_path,train_name,test_name,model_name,save_model_path,result_path)

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1test'
            model_name=k+'_'+j
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/seen/test/"+i+"/"+k+'_'+j
            save_model_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"
            train_main(trainfile_path,testfile_path,train_name,test_name,model_name,save_model_path,result_path)

            

epi ID if not found!
epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|██████████████████████████████████████| 161/161 [00:06<00:00, 24.22it/s, loss=0.651, v_num=10339]0.772488062638822
0.7794306694112609
(1,)
Train set: AUC=0.7725, AUPR=0.7794, AUC_AVG=0.7725, AUPR_AVG=0.7794
0.7501792082275232
0.7579103329327344
(1,)
Valid  set: AUC=0.7502, AUPR=0.7579, AUC_AVG=0.7502, AUPR_AVG=0.7579
Epoch 1: 100%|██████████████████████████████████████| 161/161 [00:04<00:00, 38.39it/s, loss=0.611, v_num=10339]0.7987115274071859
0.8061708439784737
(1,)
Train set: AUC=0.7987, AUPR=0.8062, AUC_AVG=0.7987, AUPR_AVG=0.8062
0.7732291463039178
0.7843308662695733
(1,)
Valid  set: AUC=0.7732, AUPR=0.7843, AUC_AVG=0.7732, AUPR_AVG=0.7843
Epoch 2: 100%|██████████████████████████████████████| 161/161 [00:04<00:00, 37.06it/s, loss=0.588, v_num=10339]0.8131585079719034
0.8214940309820067
(1,)
Train set: AUC=0.8132, AUPR=0.8215, AUC_AVG=0.8132, AUPR_AVG=0.8215
0.7837500488946536
0.7975360625482959
(1,)
Valid  set: AUC=0.7838, AUPR=0.7975, AUC_AVG=0.7838, AUPR_AVG=0.7975

0.833389705521153
0.8504103855104961
(1,)
Valid  set: AUC=0.8334, AUPR=0.8504, AUC_AVG=0.8334, AUPR_AVG=0.8504
Epoch 24: 100%|█████████████████████████████████████| 161/161 [00:06<00:00, 23.88it/s, loss=0.368, v_num=10339]
Predicting and saving results...


Predicting: 18it [00:04,  4.24it/s]


...

0.8525748459872537
0.6179612125127069
(1,)
Valid  set: AUC=0.8526, AUPR=0.618, AUC_AVG=0.8526, AUPR_AVG=0.618
Epoch 21: 100%|███████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:18<00:00, 39.18it/s, loss=0.176, v_num=10362]0.9753180522116665
0.8898709152773373
(1,)
Train set: AUC=0.9753, AUPR=0.8899, AUC_AVG=0.9753, AUPR_AVG=0.8899
0.8521956838854914
0.6188109563980362
(1,)
Valid  set: AUC=0.8522, AUPR=0.6188, AUC_AVG=0.8522, AUPR_AVG=0.6188
Epoch 22: 100%|███████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:18<00:00, 38.94it/s, loss=0.178, v_num=10362]0.9768598311265062
0.8949550973224519
(1,)
Train set: AUC=0.9769, AUPR=0.895, AUC_AVG=0.9769, AUPR_AVG=0.895
0.8526558439202788
0.6172506301322269
(1,)
Valid  set: AUC=0.8527, AUPR=0.6173, AUC_AVG=0.8527, AUPR_AVG=0.6173
Epoch 23: 100%|████████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:18<00:0

Predicting: 81it [00:19,  4.25it/s]


Done
epi ID if not found!
epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:19<00:00, 37.71it/s, loss=0.306, v_num=10363]0.7875827746146045
0.421506702324989
(1,)
Train set: AUC=0.7876, AUPR=0.4215, AUC_AVG=0.7876, AUPR_AVG=0.4215
0.7722722785331998
0.40543923258383063
(1,)
Valid  set: AUC=0.7723, AUPR=0.4054, AUC_AVG=0.7723, AUPR_AVG=0.4054
Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:18<00:00, 39.28it/s, loss=0.288, v_num=10363]0.8143642750866478
0.4890212610377679
(1,)
Train set: AUC=0.8144, AUPR=0.489, AUC_AVG=0.8144, AUPR_AVG=0.489
0.796273178531617
0.4674386755719756
(1,)
Valid  set: AUC=0.7963, AUPR=0.4674, AUC_AVG=0.7963, AUPR_AVG=0.4674
Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:18<00:00, 39.18it/s, loss=0.272, v_num=10363]0.8302004583192154
0.5323319132625695
(1,)
Train set: AUC=0.8302, AUPR=0.5

# pair300

In [7]:
pair=['more300','300','200','100','10']
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                train_name=k+'_'+j+'train'
                test_name=k+'_'+j+'test'
                model_name=k+'_'+j
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/10cross_validation/pair300/"+l+"/"+i+"/"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/10cross_validation/pair300/more300/"+i+"/"
                save_model_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair300/seen/"+l+"/"+i+"/"
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                train_main(trainfile_path,testfile_path,train_name,test_name,model_name,save_model_path,result_path)
            


epi ID if not found!
epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|███████████████████████████████████████████| 150/150 [00:03<00:00, 38.17it/s, loss=0.658, v_num=12409]0.7816376069060016
0.7875140710858045
(1,)
Train set: AUC=0.7816, AUPR=0.7875, AUC_AVG=0.7816, AUPR_AVG=0.7875
0.7672008701487623
0.7638095119697406
(1,)
Valid  set: AUC=0.7672, AUPR=0.7638, AUC_AVG=0.7672, AUPR_AVG=0.7638
Epoch 1: 100%|███████████████████████████████████████████| 150/150 [00:03<00:00, 38.94it/s, loss=0.604, v_num=12409]0.805579073964935
0.8145868016253139
(1,)
Train set: AUC=0.8056, AUPR=0.8146, AUC_AVG=0.8056, AUPR_AVG=0.8146
0.7839178898205795
0.7864986178076467
(1,)
Valid  set: AUC=0.7839, AUPR=0.7865, AUC_AVG=0.7839, AUPR_AVG=0.7865
Epoch 2: 100%|███████████████████████████████████████████| 150/150 [00:04<00:00, 36.27it/s, loss=0.578, v_num=12409]0.8161545697705358
0.8257981923457716
(1,)
Train set: AUC=0.8162, AUPR=0.8258, AUC_AVG=0.8162, AUPR_AVG=0.8258
0.791265778790391
0.7943386974690159
(1,)
Valid  set: AUC=0.7913, AUPR=0.7943, AUC_AVG=0.7913, A

Predicting: 7it [00:02,  2.75it/s]


Done
epi ID if not found!
epi ID if not found!


...

Epoch 24: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.93it/s, loss=0.747, v_num=12433]
Predicting and saving results...


Predicting: 7it [00:05,  1.33it/s]


Done
epi ID if not found!
epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|█████████████████████████████████████████████| 150/150 [00:04<00:00, 31.57it/s, loss=0.7, v_num=12434]0.7500779489187145
0.7505496999677463
(1,)
Train set: AUC=0.7501, AUPR=0.7505, AUC_AVG=0.7501, AUPR_AVG=0.7505
0.733975516956036
0.7314790761529332
(1,)
Valid  set: AUC=0.734, AUPR=0.7315, AUC_AVG=0.734, AUPR_AVG=0.7315
Epoch 1: 100%|███████████████████████████████████████████| 150/150 [00:04<00:00, 33.18it/s, loss=0.646, v_num=12434]0.7678030428755837
0.7726310110762284
(1,)
Train set: AUC=0.7678, AUPR=0.7726, AUC_AVG=0.7678, AUPR_AVG=0.7726
0.7414901259403752
0.7489243338416844
(1,)
Valid  set: AUC=0.7415, AUPR=0.7489, AUC_AVG=0.7415, AUPR_AVG=0.7489
Epoch 2:  29%|████████████▌                               | 43/150 [00:01<00:03, 33.51it/s, loss=0.628, v_num=12434]

# Validation

In [10]:
def Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path):
    test_name=test_name
    train_name=train_name
    class Config:
        def __init__(self, path, file_list):
            self.path = path
            self.file_list = file_list
    train_set = SeqLevelDataset(Config(path=trainfile_path, file_list=[train_name]))
    val_set = SeqLevelDataset(Config(path=testfile_path, file_list=[test_name]))
    ckpt_path = model_path+'_model.pth'
    model = SeqLevelSystem(config, train_set, val_set)
    model.load_state_dict(torch.load(ckpt_path))
    print('Predicting val...')
    results = model.predict(model.val_dataloader())
    print('Saving results...')
    if len(results) == 4:
        pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob']).to_csv(result_path+'probability.csv', index=False)
    else:
        aa=pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob', 'epi_id'])
        aa=aa[['CDR3B', 'Epitope', 'y_true', 'y_prob']]
        aa['y_pred'] = aa['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
        aa.to_csv(result_path+'probability.csv', index=False)
    print('Done')

In [8]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1Validation'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/Validation/"+i+"/"
            model_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/seen/validation/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1Validation'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/Validation/"+i+"/"
            model_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/seen/validation/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 

epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 10it [00:03,  3.02it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 10it [00:03,  2.83it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


...

Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 10it [00:03,  2.79it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 10it [00:03,  2.98it/s]


Saving results...
Done


# unknown-test

In [13]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1test'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/unknow/"+i+"/"
            modela_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/unseen/test/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 
            

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1test'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/unknow/"+i+"/"
            model_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+"_"+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/unseen/test/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 

epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 14it [00:04,  2.97it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 14it [00:05,  2.78it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


...

Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 14it [00:04,  2.96it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 14it [00:04,  2.91it/s]


Saving results...
Done


# unknown-validaiton

In [5]:
def fix_name(data_path,save_path):
    data=pd.read_csv(data_path)
    data=data[['CDR3B','Epitope','Affinity']]
    data.rename(columns={'CDR3B': 'cdr3', 'Epitope': 'epi', 'Affinity': 'y_true'}, inplace=True)
    data.to_csv(save_path) 
    
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name1:
        data_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/validation/"+i+"/" +j+"_"'1_1validation.csv'               
        save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/unknow/validation/"+i+"/" +j+"_"+'1_1validation.csv'   
        fix_name(data_path,save_path)



In [8]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1validation'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/unknow/validation/"+i+"/"
            model_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 
            

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            train_name=k+'_'+j+'train'
            test_name=k+'_'+'1_1validation'
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/pair50/"+i+"/"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/data/Retrain/unknow/validation/"+i+"/"
            model_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/model/Retrain/pair50/"+i+"/"+k+"_"+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            Validation_main(trainfile_path,testfile_path,train_name,test_name,model_path,result_path) 

epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 6it [00:04,  1.26it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 6it [00:04,  1.36it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


...

Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 6it [00:03,  1.95it/s]


Saving results...
Done
epi ID if not found!
epi ID if not found!
Predicting val...


Predicting: 6it [00:03,  1.98it/s]


Saving results...
Done


# TCR numbers top5(Parameter Adjustment)

In [7]:
def fix_name(data_path,save_path):
    data=pd.read_csv(data_path)
    data=data[['CDR3B','Epitope','Affinity']]
    data.rename(columns={'CDR3B': 'cdr3', 'Epitope': 'epi', 'Affinity': 'y_true'}, inplace=True)
    data.to_csv(save_path) 
    
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
for i in reap:
    for j in name:
        for k in number:
            data_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'               
            save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/top5/healthy/"+i+"_" +j+"_"+k+'.csv'   
            fix_name(data_path,save_path)

for i in reap:
    for j in name:
        data_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+'test.csv'             
        save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/top5/healthy/"+i+"_" +j+"_"+'test.csv'
        fix_name(data_path,save_path)

In [11]:
def train_main(trainfile_path, testfile_path, train_name, test_name, epochs,result_path):
    class Config:
        def __init__(self, path, file_list):
            self.path = path
            self.file_list = file_list

    config_path = 'configs/seqlevel_all.yml'
    config = load_config(config_path)  

    train_set = SeqLevelDataset(Config(path=trainfile_path, file_list=[train_name])) 
    val_set = SeqLevelDataset(Config(path=testfile_path, file_list=[test_name])) 

    model = SeqLevelSystem(config, train_set, val_set)
    checkpoint = ModelCheckpoint(monitor='valid/auc_avg', save_last=True, mode='max', save_top_k=1)
    earlystop = EarlyStopping(monitor='valid/auc_avg', patience=15, mode='max')
    trainer = pl.Trainer(
        max_epochs=epochs,
        gpus=1,
        callbacks=[checkpoint, earlystop],
        default_root_dir=os.path.join(os.getcwd(), 'logs', config.name)
    )

    trainer.fit(model)
    shutil.copy2(config_path, os.path.join(trainer.log_dir, os.path.basename(config_path)))

    print('Predicting and saving results...')
    results = model.predict(model.val_dataloader())
    columns = ['CDR3B', 'Epitope', 'y_true', 'y_prob']
    if len(results) == 4:
        pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob']).to_csv(result_path+'probability.csv', index=False)
    else:
        aa= pd.DataFrame(zip(*results), columns=['CDR3B', 'Epitope', 'y_true', 'y_prob', 'epi_id'])
        aa=aa[['CDR3B', 'Epitope', 'y_true', 'y_prob']]
        aa.to_csv(result_path+'probability.csv', index=False)
    print('Done')


In [12]:
import os
GPU_NUMBER = [1]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO"

In [ ]:
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
epochs=[30,50,70,90,100]
for i in reap:
    for j in name:
        for k in number:
            for epoch  in epochs:
                train_name=i+"_" +j+"_"+k
                test_name=i+"_" +j+"_"+'test' 
                trainfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/train_teim/data/top5/healthy/"     
                testfile_path=trainfile_path
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/result/top5/seen/healthy/"+i+"_" +j+"_"+k+"_"+  str(epoch)
                train_main(trainfile_path,testfile_path,train_name,test_name,epoch,result_path) 

epi ID if not found!
epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|██████████████████████████████████| 12/12 [00:00<00:00, 29.68it/s, loss=1.05, v_num=10408]0.7372487777777779
0.7301218186175829
(1,)
Train set: AUC=0.7372, AUPR=0.7301, AUC_AVG=0.7372, AUPR_AVG=0.7301
0.7152639999999999
0.7086120642534485
(1,)
Valid  set: AUC=0.7153, AUPR=0.7086, AUC_AVG=0.7153, AUPR_AVG=0.7086
Epoch 1: 100%|█████████████████████████████████| 12/12 [00:00<00:00, 23.74it/s, loss=0.908, v_num=10408]0.787739388888889
0.7921941495629433
(1,)
Train set: AUC=0.7877, AUPR=0.7922, AUC_AVG=0.7877, AUPR_AVG=0.7922
0.7584839999999999
0.7573006218151987
(1,)
Valid  set: AUC=0.7585, AUPR=0.7573, AUC_AVG=0.7585, AUPR_AVG=0.7573
Epoch 2: 100%|█████████████████████████████████| 12/12 [00:00<00:00, 22.47it/s, loss=0.824, v_num=10408]0.8065687222222222
0.8128166988034393
(1,)
Train set: AUC=0.8066, AUPR=0.8128, AUC_AVG=0.8066, AUPR_AVG=0.8128
0.7774480000000001
0.7800983640986588
(1,)
Valid  set: AUC=0.7774, AUPR=0.7801, AUC_AVG=0.7774, AUPR_AVG=0.7801
Epoch 3: 100%|██████

Epoch 26: 100%|████████████████████████████████| 12/12 [00:00<00:00, 32.38it/s, loss=0.428, v_num=10408]0.9500055555555557
0.9521356305412647
(1,)
Train set: AUC=0.95, AUPR=0.9521, AUC_AVG=0.95, AUPR_AVG=0.9521
0.853084
0.8610401692831071
(1,)
Valid  set: AUC=0.8531, AUPR=0.861, AUC_AVG=0.8531, AUPR_AVG=0.861
Epoch 27: 100%|████████████████████████████████| 12/12 [00:00<00:00, 31.16it/s, loss=0.416, v_num=10408]0.9536600000000001
0.9561283438789743
(1,)
Train set: AUC=0.9537, AUPR=0.9561, AUC_AVG=0.9537, AUPR_AVG=0.9561
0.8543999999999998
0.8633556939555298
(1,)
Valid  set: AUC=0.8544, AUPR=0.8634, AUC_AVG=0.8544, AUPR_AVG=0.8634
Epoch 28: 100%|████████████████████████████████| 12/12 [00:00<00:00, 33.51it/s, loss=0.411, v_num=10408]0.9552345555555556
0.9575855917913322
(1,)
Train set: AUC=0.9552, AUPR=0.9576, AUC_AVG=0.9552, AUPR_AVG=0.9576
0.8546639999999999
0.8665925807589056
(1,)
Valid  set: AUC=0.8547, AUPR=0.8666, AUC_AVG=0.8547, AUPR_AVG=0.8666
Epoch 29: 100%|████████████████████

Predicting: 2it [00:00,  2.17it/s]

...

Done
epi ID if not found!


epi ID if not found!


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name     | Type | Params
----------------------------------
0 | teim_seq | TEIM | 1.3 M 
----------------------------------
1.2 M     Trainable params
38.2 K    Non-trainable params
1.3 M     Total params
5.012     Total estimated model params size (MB)
/home/luyanping/miniconda3/envs/teim/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck

Epoch 0: 100%|█████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 34.37it/s, loss=1.07, v_num=10427]0.7288327256944445
0.7183186442807481
(1,)
Train set: AUC=0.7288, AUPR=0.7183, AUC_AVG=0.7288, AUPR_AVG=0.7183
0.71348
0.7111124190726945
(1,)
Valid  set: AUC=0.7135, AUPR=0.7111, AUC_AVG=0.7135, AUPR_AVG=0.7111
Epoch 1: 100%|████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 35.07it/s, loss=0.972, v_num=10427]0.7725545138888888
0.7706330899988686
(1,)
Train set: AUC=0.7726, AUPR=0.7706, AUC_AVG=0.7726, AUPR_AVG=0.7706
0.7510159999999999
0.7493951111521474
(1,)
Valid  set: AUC=0.751, AUPR=0.7494, AUC_AVG=0.751, AUPR_AVG=0.7494
Epoch 2: 100%|████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 36.09it/s, loss=0.842, v_num=10427]0.7983203125000001
0.80190515074903
(1,)
Train set: AUC=0.7983, AUPR=0.8019, AUC_AVG=0.7983, AUPR_AVG=0.8019
0.7683679999999999
0.7695841426029408
(1,)
Valid  set: AUC=0.7684

Epoch 24: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 34.98it/s, loss=0.429, v_num=10427]0.9458737847222223
0.9486991842051081
(1,)
Train set: AUC=0.9459, AUPR=0.9487, AUC_AVG=0.9459, AUPR_AVG=0.9487
0.85328
0.8598239209527616
(1,)
Valid  set: AUC=0.8533, AUPR=0.8598, AUC_AVG=0.8533, AUPR_AVG=0.8598
Epoch 25: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 36.17it/s, loss=0.416, v_num=10427]0.9502970486111111
0.9534156344598862
(1,)
Train set: AUC=0.9503, AUPR=0.9534, AUC_AVG=0.9503, AUPR_AVG=0.9534
0.854824
0.8629619400373773
(1,)
Valid  set: AUC=0.8548, AUPR=0.863, AUC_AVG=0.8548, AUPR_AVG=0.863
Epoch 26: 100%|███████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 32.70it/s, loss=0.407, v_num=10427]0.9520743923611111
0.9552648444762699
(1,)
Train set: AUC=0.9521, AUPR=0.9553, AUC_AVG=0.9521, AUPR_AVG=0.9553
0.853256
0.8621316441963052
(1,)
Valid  set: AUC=0.8533, AUPR=0.8621, AUC